## Analise de dados de vendas de carros da BMW

## Descrição:

Dados de vendas da BMW que contém registros detalhados das vendas de carros da marca, oferecendo informações valiosas sobre o desempenho de vendas, preferências dos clientes e tendências do mercado. Este conjunto de dados é essencial para analisar padrões de vendas, prever a demanda, avaliar a popularidade de diferentes modelos e apoiar estratégias empresariais no setor automotivo.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")


## Coletando dados

In [ ]:
project_id = 'fiap-tc03'
dataset_id = 'vendas'
tabela_name= 'tb_vendas'

In [ ]:
df_raw = pd.read_gbq(f"SELECT * FROM {dataset_id}.{tabela_name}", project_id=project_id, dialect='standard')

In [ ]:
df_raw.head()

In [ ]:
df_raw.tail()

In [ ]:
df_raw.info()

In [ ]:
df_raw.dtypes

In [ ]:
df_raw.shape

In [ ]:
df_raw.isnull().sum()

In [ ]:
df_raw.duplicated().sum()

In [ ]:
df_raw.describe()

In [ ]:
df_raw.columns

# Salvar como tabela no Bigquery

In [ ]:
df_raw.to_gbq(
    destination_table=f"{dataset_id}.{tabela_name}_refined",  # Exemplo: 'meu_dataset.minha_tabela'
    project_id=project_id,
    if_exists='replace'
)


# Carregar os dados da refined

In [ ]:
df = pd.read_gbq(f"SELECT * FROM {dataset_id}.{tabela_name}_refined", project_id=project_id, dialect='standard')


## Data visualizations

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(data=df, x="Year", y="Sales_Volume", estimator="sum", ci=None)
plt.title("Total de volume de vendas por ano", fontsize=14, weight="bold")
plt.ylabel("Total de vendas")
plt.xlabel("Anos")
plt.xticks(rotation=45)
plt.show()


In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(data=df, x="Region", order=df["Region"].value_counts().index)
plt.title("Distribuição regional de vendas dos carros", fontsize=14, weight="bold")
plt.ylabel("Registros")
plt.xlabel("Região")
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
sns.histplot(df["Price_USD"], bins=30, kde=True)
plt.title("Distribuição de preços dos carros", fontsize=14, weight="bold")
plt.xlabel("Preços (USD)")
plt.ylabel("Freqüência")
plt.show()


In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(data=df, x="Fuel_Type", hue="Transmission")
plt.title("Tipo de combustível vs Tipo de transmissão", fontsize=14, weight="bold")
plt.xlabel("Tipo de combustível")
plt.ylabel("Total")
plt.show()


In [ ]:
plt.figure(figsize=(10,6))
sns.scatterplot(data=df, x="Mileage_KM", y="Price_USD", hue="Sales_Classification", alpha=0.6)
plt.title("KM vs Cor by Classificacao da venda", fontsize=14, weight="bold")
plt.xlabel("KM")
plt.ylabel("Preço (USD)")
plt.ylim(0, df["Price_USD"].quantile(0.95))
plt.show()


In [ ]:
plt.figure(figsize=(8,6))
sns.countplot(data=df, x="Sales_Classification", order=df["Sales_Classification"].value_counts().index)
plt.title("Vendas de carro e classificacao Breakdown", fontsize=14, weight="bold")
plt.ylabel("Quantidades")
plt.xlabel("Classificacao da vendas")
plt.show()


In [ ]:
plt.figure(figsize=(10,6))
corr = df[["Year", "Engine_Size_L", "Mileage_KM", "Price_USD", "Sales_Volume"]].corr()
sns.heatmap(corr, annot=True, cmap="Blues", fmt=".2f")
plt.title("Mapa de calor de correlação de recursos numéricos", fontsize=14, weight="bold")
plt.show()

## Predictive modeling

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier

In [ ]:
categorical_cols = ["Model", "Region", "Color", "Fuel_Type", "Transmission"]

for col in categorical_cols:
    encoder = LabelEncoder()

    df[col] = encoder.fit_transform(df[col].astype(str))

    mapping_df = pd.DataFrame({
        col: encoder.classes_,                  # nomes originais (strings)
        f"{col}_encoded": range(len(encoder.classes_))  # números que o LabelEncoder atribuiu
    })

    mapping_df.to_csv(f"{col}_label_mapping.csv", index=False)

    table_label_id = f"{project_id}.{dataset_id}.{col.lower()}_label_mapping"
    print(table_label_id)

    mapping_df.to_gbq(
        destination_table=table_label_id,
        if_exists='replace'
    )

In [ ]:
X = df.drop(["Sales_Classification","Sales_Volume"], axis=1)
y = df["Sales_Classification"]

In [ ]:
X.columns

In [ ]:
if y.dtype == 'object' or y.dtype == 'category':
    y = encoder.fit_transform(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=51, stratify=y
)

In [ ]:
## Comparação de Modelos de Classificação para Vendas BMW

In [ ]:
models = {
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Gradient Boosting": GradientBoostingClassifier(),
    "AdaBoost": AdaBoostClassifier(),
    "KNN": KNeighborsClassifier(),
    "SVM": SVC(),
    "Naive Bayes": GaussianNB(),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric="mlogloss"),
    "Logistic Regression": LogisticRegression(max_iter=2000)
}

In [ ]:
results = []

for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    acc = accuracy_score(y_test, preds) * 100
    results.append((name, acc))

In [ ]:
results_df = pd.DataFrame(results, columns=["Model", "Accuracy (%)"])
print(results_df.sort_values(by="Accuracy (%)", ascending=False).reset_index(drop=True))

## Salvar o modelo em pkl

In [ ]:
import joblib
modelo_escolhido="Random Forest"
arquivo_modelo="modelo_bmw.pkl"
joblib.dump(models[modelo_escolhido], arquivo_modelo)

# Mover o modelo para o bucket

In [ ]:
bucket_name = "gcs-raw-fiap-a2555e1b"

### Backup do modelo anterior

In [ ]:
from google.cloud import storage
from google.api_core.exceptions import NotFound

def mover_arquivo(bucket_name, caminho_origem, caminho_destino):
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob_origem = bucket.blob(caminho_origem)

    if not blob_origem.exists():
        print(f"Arquivo '{caminho_origem}' não encontrado no bucket '{bucket_name}'.")
        return

    # Copia o arquivo para o novo local
    bucket.copy_blob(blob_origem, bucket, caminho_destino)

    # Remove o arquivo original
    blob_origem.delete()

    print(f"Arquivo movido de {caminho_origem} para {caminho_destino}")


In [ ]:
import time

timestamp_segundos = int(time.time())

mover_arquivo(
    bucket_name,
    f"modelo/{modelo_escolhido}/model.pkl",
    f"modelo/{modelo_escolhido}/bkp/model-{timestamp_segundos}.pkl"
)

### Copiar o novo modelo para o bucket para ser usado.

In [ ]:
from google.cloud import storage
import os

def mover_arquivo_local_para_bucket(bucket_name, caminho_local, caminho_bucket):
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(caminho_bucket)

    blob.upload_from_filename(caminho_local)
    print(f"Arquivo '{caminho_local}' enviado para 'gs://{bucket_name}/{caminho_bucket}'.")

    os.remove(caminho_local)
    print(f"Arquivo local '{caminho_local}' removido.")


In [ ]:
mover_arquivo_local_para_bucket(
    bucket_name,
    arquivo_modelo,
    f"modelo/{modelo_escolhido}/model.pkl"
)